# Raku RAG demo

<span style="font-size: 16pt;">... over </span>
<span style="font-size: 16pt; font-style: italic;">Economics articles and interview transcripts</span>

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
[EconomicsAI at GitHub](https://github.com/antononcube/EconomicsAI)      
April 2025 


-----

## Introduction

The so called ***Retrieval Augmented Generation (RAG)*** functionalities of Large Language Models (LLMs) are demonstrated using semantic nearest neighbors graphs.

Vector Databases (VDBs) made with the Raku package ["LLM::RetrievalAugmentedGeneration"](https://raku.land/zef:antononcube/LLM::RetrievalAugmentedGeneration) are used.

1. Semantic graph for words 
    - Select a set of words
    - Find LLM embeddings for the those words
    - Find nearest neighbor graph for the obtained vectors

2. Nearest neighbor graph -- joined VDBs
    - Ingest VDBs corresponding to (long) podcast interviews
        - "Modern Wisdom" with [Eric Weinstein](https://en.wikipedia.org/wiki/Eric_Weinstein)
        - Eclectic and long interviews (≈3.5 hours each)
        - Heterogeneous content (wide variety of topics)
        - *(Hence, suitable for semantic similarities demos)*
    - Merge VDBs
    - Make a corresponding semantic graph
        - Nearest neighbor graph
        - Connected components

3. Nearest neighbor graph and RAG 
    - Create a semantic relationship graph for an interview
    - LLM-derivation of summaries for top connected components
    - Graph plot traces 
        - Using suitable graph tooltips
    - Do a retrieval based LLM summary for a query

-----

## Setup

In [1]:
use LLM::RetrievalAugmentedGeneration;
use LLM::RetrievalAugmentedGeneration::VectorDatabase;

use XDG::BaseDirectory :terms;

use Data::Importers;
use Data::Reshapers;
use Data::Summarizers;
use Math::Nearest;
use Math::DistanceFunctions::Native;
use Statistics::OutlierIdentifiers;

use Math::Nearest;
use Graph;

use JavaScript::D3;

### JavaScript

Here we prepare the notebook to visualize with JavaScript:

In [2]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

Verification:

In [3]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

Here we set a collection of visualization variables:

In [4]:
my $title-color = 'Ivory';
my $stroke-color = 'SlateGray';
my $tooltip-color = 'LightBlue';
my $tooltip-background-color = 'none';
my $background = '1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $vertex-size = 6;
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

{charge => {iterations => 4, strength => -30}, collision => {iterations => 4, radius => 50}, link => {distance => 30}}

### LLM configurations

In [22]:
sink my $conf41-mini = llm-configuration('ChatGPT', model => 'gpt-4.1-mini', temperature => 0.45);
sink my $conf41 = llm-configuration('ChatGPT', model => 'gpt-4.1', temperature => 0.45);
sink my $conf51 = llm-configuration('ChatGPT', model => 'gpt-5.1', reasoning-effort => 'none');
sink my $conf51-codex = llm-configuration('ChatGPT', path => 'responses', model => 'gpt-5.1-codex');
sink my $conf51-codex-mini = llm-configuration('ChatGPT', path => 'responses', model => 'gpt-5.1-codex-mini');
sink my $conf-gemini20-flash = llm-configuration('Gemini', model => 'gemini-2.0-flash');
sink my $conf-gemini25-flash = llm-configuration('Gemini', model => 'gemini-2.5-flash');
sink my $conf-gemini25-pro = llm-configuration('Gemini', model => 'gemini-2.5-pro');

------

## Ingest vector databases

Ingest vector database from the default directory and show a summary:

In [5]:
#% html
my @field-names = <id name item-count dimension version llm-service llm-embedding-model created>;

my $vdbDirName = $*HOME ~ '/.local/share/raku/LLM/SemanticSearchIndex';
vector-database-objects($vdbDirName, f=>'hash', :flat)

==> { $_.map({ $_<created> = $_<file>.IO.created.DateTime.Str.subst('T',' ').substr(^19); $_}).sort(*<created>).reverse }()
==> to-html(:@field-names)

id,name,item-count,dimension,version,llm-service,llm-embedding-model,created
e1de43bc-0d12-4842-98a7-683321534396,words,20,1536,0,chatgpt,text-embedding-3-small,2025-10-05 22:01:08
2ea442a9-6ca5-4c3d-adfb-b5206dc930e9,words,20,1536,0,chatgpt,text-embedding-3-small,2025-10-05 21:59:07
9794ad64-0602-45b5-95e6-f7f699cd4505,words,20,1536,0,chatgpt,text-embedding-3-small,2025-10-05 21:56:57
4da2948b-3252-4be0-ae66-e3dacce6b545,Raku-study-group,196,2048,0,LLaMA,llama-embedding,2025-10-05 19:58:13
c1261e9f-535f-4721-8984-5a8a1331efb3,H2O.ai-LLMs-course,122,2048,0,LLaMA,llama-embedding,2025-10-05 13:59:41
90c0b166-e111-4f91-9924-6e402becf3f2,words,20,1536,0,chatgpt,text-embedding-3-small,2025-09-09 18:52:26
8b1531c4-58f5-4f67-a7e6-14196df63acb,words,20,768,0,gemini,embedding-001,2025-08-05 21:29:42
67f751f8-853a-4545-9d9e-d7a72b7a9b4b,words,20,768,0,gemini,embedding-001,2025-07-23 16:33:02
6cc84d8f-5a8f-47b7-95e2-6008ef2e4d03,words,20,768,0,gemini,embedding-001,2025-07-17 21:26:14
c4cd86f8-8908-44a4-8a39-9f9e476cba05,EconomicsAI,1352,2048,0,llama,llama-embedding,2025-05-03 23:00:23


Get specific databases:

In [7]:
my @vdbs = vector-database-objects($vdbDirName, f=>'hash', :flat).grep({ $_<id> ∈ <c4cd86f8-8908-44a4-8a39-9f9e476cba05> }).map({ create-vector-database(file => $_<file>) })

[VectorDatabase(:id("c4cd86f8-8908-44a4-8a39-9f9e476cba05"), :name("EconomicsAI"), :elements(1352), :sources(76), :precision(num64))]

Sample vectors:

In [8]:
.say for @vdbs.head.vectors.pick(3)

nrPiSm2AZ54.txt.7 => NativeCall::Types::CArray[num64].new
e3qLvgyRSKc.txt.1 => NativeCall::Types::CArray[num64].new
i7iGPm0vloA.txt.9 => NativeCall::Types::CArray[num64].new


Sample items:

In [9]:
#% html
@vdbs.head.items.pick(3).map({ <key value> Z=> $_.kv })».Hash.Array 
==> to-html(field-names => <key value>, align => 'left')

key,value
LAagE-7BLN4.txt.3,employer for Americans This is an unheard of luxury in Europe it's taken for granted as our universities that do not have any tuition as our national health care services that are provided as a matter of Human Rights and as a ban on capital punishment as another human right I could go on but that's real I don't want to take that away from those that's a kind of socialism now there's a second kind I'm going to call it the Soviet socialism here we go very far in the direction of the government taking over not just regulating as they do in Europe not just helping with certain social services but literally uh in industry and to an considerable extend in agriculture as well the government becomes the owner operator of the Enterprise and that has a long history I won't go into here and here's the third the third is a hybrid of the first two that's what I would call the People's Republic of China a roughly 5050 split between private Enterprises regulated by a government and public Enterprises owned and operated by the same government difficult to manage so that you have a very powerful uh Communist Party sitting a top all of this with its entire government apparatus to manage these are different Notions of what socialism is and what we know from the evidence is if you're measuring in terms of economic growth and I'm not arguing that that's the right measure or the only measure or the best measure but if you use that measure then it's crystal clear to me that the Chinese variant has been more successful at solving that problem of economic growth than capital ISM ever did or than the other two kinds of socialism ever did and that the whole world is now confronted with what I just said what are you going to do when you have a system like in China which has been able for the last 30 years to grow their economy 5 to % per year which is three times what the West is able to do which continues to right now and shows no sign of going away I mean the this is the enormity and the Europeans are the are the laggards in all of this their socialism is the weakest in terms of economic growth their socialism is the most broken apart into uh fights between small units I mean they can't get the Italians together with the belgians I mean this is bizarre what what they are unable to do you have parts of them that are so broken down and I'm thinking here of uh the United Kingdom they're so broken down that not only can they not question their pathetic capitalist system but they actually decided that what was wrong there was nothing to do with
Uz5-PtkUw9s.txt.18,take two roads at once is going to have a broken hip joint uh that's basically what we have there so uh if the United States cannot even admit Palestine as a member uh what on Earth can it do it it was the United Nations that created Israel and it itself Bears the responsibility for recognizing Israel and endorsing its explicit aim of genocide against Palestinians from the new settler country's Inception in 8 1948 the United States I'm sorry the United Nations accepted the settler State even as the uh the stern gang was killing all the Palestinians to uh to let its uh its Zionist followers come in and uh the United Nations was powerless to stop it and the United Nations is powerless to act in the very way it's constructed with a security Council that can be blocked by the United States uh and where you can have votes to condemn Israel by the uh uh the only two countries opposing are the United States Israel and a few Pacific island countries uh the whole rest of the world is against them and cannot do anything it's obvious that if there is going to be any way of preventing what we're describing this attack on civilization there has to be a new alternative to the United Nations uh and that alternative has to have a military enforcement law arm of inter interational law and it has to uh realize that this is an existential issue that requires its own ideological Doctrine uh

-----

## Combined databases graph

Here we join the selected databases:

In [10]:
#my $vdbObj2 = vector-database-join(@vdbs)
my $vdbObj2 = @vdbs.head


VectorDatabase(:id("c4cd86f8-8908-44a4-8a39-9f9e476cba05"), :name("EconomicsAI"), :elements(1352), :sources(76), :precision(num64))

Find a nearest neighbors graph:

In [11]:
my @edges2 = nearest-neighbor-graph($vdbObj2.vectors.pairs, 1, method => 'Scan', distance-function => &euclidean-distance, format => 'raku');

my $gr2 = Graph.new(@edges2)

Graph(vertexes => 1352, edges => 997, directed => False)

In [12]:
my @comps2 = $gr2.connected-components.sort(-*.elems).head(8);

@comps2.elems

8

In [13]:
#% js
@edges2
==> js-d3-graph-plot(
        :$background,
        highlight => @comps2.map({[|$_, |$gr2.subgraph($_).edges]}),
        vertex-label-color => 'none',
        edge-thickness => 2,
        vertex-size => 4,
        vertex-color => 'Blue',
        width => 1100,
        height => 950,
        edge-color => 'Gray',
        vertex-color => 'DarkGray',
        force => {charge => {strength => -10, iterations => 4}, collision => {radius => 14, iterations => 4}, link => {distance => 1};}
    )

-----

## Nearest neighbors for a query

Query:

In [14]:
#my $query = 'What are the effects of the tariffs on deficit countries and on surplus countries?';
my $query = 'What are the problems with the newly introduced tariffs? Give a list of bullet points.'
#my $query = 'What the professors think about Marxism teachings in USA? Give a list of bullet points.'

What are the problems with the newly introduced tariffs? Give a list of bullet points.

In [15]:
# Same configuration is used for the vector database creation
#my $conf = llm-configuration("Gemini");
my $conf = llm-configuration(
    'LLaMA', 
    model => 'llama-embedding',
    base-url => 'http://127.0.0.1:8080',
    embedding-model => 'llama-embedding',
    embedding-function => &WWW::LLaMA::Embeddings::LLaMAEmbeddings,
    module => 'WWW::LLaMA'
    );
$conf.Hash.elems

26

Make a query vector:

In [16]:
my $vec = llm-embedding($query, e => $conf).head».Num.Array;

deduce-type($vec)

Vector(Atom((Numeric)), 2048)

Find nearest neighbors, plot, and determine outliers:

In [17]:
my @res = $vdbObj2.nearest($vec, 100, distance-function => &euclidean-distance, prop => <label distance>);

deduce-type(@res)

Vector(Tuple([Atom((Str)), Atom((Numeric))]), 100)

In [18]:
#% js
my @points = @res».tail;
my @data = @points.kv.map( -> $x, $y { %(:$x, :$y, group => 'data' ) });
my @pos = |@points[outlier-identifier(@points, identifier => &quartile-identifier-parameters)];
@data.append(@pos.kv.map({ %(x => $_, y => @points[$_], group => 'outliers' ) }) );

js-d3-list-plot(@data, background => '#1F1F1F', :600width, :300height, :grid-lines, title => "{@pos.elems} outliers", title-color => 'Gray')

Find the nearest neighbors:

In [19]:
my @nns = |$vdbObj2.nearest($vec, 8, distance-function => &euclidean-distance).flat(:hammer)

[_igKkNlIc9c.txt.17 w--fsqQQQa0.txt.15 FW0fFJJSsW8.txt.19 ke9UvCezqIo.txt.8 e5Ed7b1-LnY.txt.1 Uz5-PtkUw9s.txt.14 hzIft13pASY.txt.5 cgQfWAuyTtk.txt.39]

Get the corresponding "paragraphs":

In [20]:
my @paragraphs = $vdbObj2.items{|@nns};

.say for @paragraphs».&text-stats

(chars => 2552 words => 468 lines => 1)
(chars => 1119 words => 208 lines => 1)
(chars => 2559 words => 483 lines => 1)
(chars => 2559 words => 490 lines => 1)
(chars => 2558 words => 494 lines => 1)
(chars => 2556 words => 461 lines => 1)
(chars => 2559 words => 446 lines => 1)
(chars => 2558 words => 462 lines => 1)


Generate LLM based answer using the nearest neighbors:

In [23]:
#% markdown

llm-synthesize([ 
    "Summarize the following text into a list of three-four points, each with no more than 8 words:",
    $vdbObj2.items{|@nns.sort}.join(" "),
], e => $conf41-mini)

- Europe faces irreversible economic decline from sanctions.  
- US-Europe conflict harms German industry and jobs.  
- Global struggle: US/NATO vs. BRICS and China.  
- Need new narratives to challenge dominant economic views.

In [25]:
#% markdown

llm-synthesize([ 
    "Answer the inquiry:",
    $query,
    "using the following text:",
    $vdbObj2.items{|@nns.sort}.join(" "),
], e => $conf41-mini)

Problems with the newly introduced tariffs based on the provided text:

- Volkswagen and other German companies face significant challenges competing in electric cars with China without imposing huge tariffs on Chinese imports.
- The tariffs and related economic measures reverberate negatively throughout the German and broader European economy.
- Many small and medium-sized German firms are going out of business, and once lost, these firms cannot be remade.
- German industrialists and export industries heavily depend on trade with China, and tariffs threaten these economic relationships.
- The sanctions and tariffs contribute to deindustrialization in Germany, with factories closing and thousands of workers laid off.
- Europe's refusal to buy cheap Russian oil and gas (due to sanctions) forces them to buy more expensive energy on the world market, increasing costs for industries.
- The tariffs and sanctions create collateral damage in Europe, harming European economies while the U.S. maintains its freedom of action.
- The economic and trade restrictions contribute to a broader geopolitical struggle and global fracture, pitting NATO and Western countries against much of the rest of the world.
- The tariffs exacerbate internal conflicts within countries, such as between corporations wanting continued trade with China and political forces pushing for hostility.
- These measures are part of a broader pattern of miscalculations and denial regarding the global economic and political reality, risking long-term damage.
- The tariffs and sanctions contribute to industrial decline and loss of competitiveness in Europe, particularly Germany, which has not been seen for decades.
- The economic disruption caused by tariffs and sanctions is difficult to reverse and may lead to irreversible damage to key industries and economic sectors.

In [26]:
#llm-prompt-data(/:i SophisticatedFeedback /)
llm-prompt-data(/:i Propa /)

{FindPropagandaMessage => Finds hidden (propaganda) messages in texts}

-----

## Extract wisdom

Focus text:

In [27]:
my $txtFocus = $vdbObj2.items{|@nns.sort}.join(" ");

text-stats($txtFocus)

(chars => 19027 words => 3512 lines => 1)

### Tabulate topics

In [ ]:
#% html
my $tblThemes = llm-synthesize(llm-prompt("ThemeTableJSON")($txtFocus, "article", 30), e => $conf41-mini, form => sub-parser('JSON'):drop);
$tblThemes ==> data-translation(field-names=><theme content>, align => 'left')

theme,content
European Economic Challenges and Industrial Decline,"Volkswagen's struggles, German industry layoffs, impact of sanctions on energy supply, and irreversible damage to Europe's industrial base."
Geopolitical Tensions and US-Europe Relations,"US actions causing irreversibilities for Europe, seizure of Russian assets, and Europe's collateral damage amid US freedom of action."
Rise of Nationalistic Parties in Europe,"Nationalistic parties gaining power over socialist ones, and far-right tendencies within parties labeled socialist in Europe."
Legacy of Cold War and Demonization of Left,"Post-WWII demonization of leftists, socialists, and communists, and its impact on global politics and conflicts."
Ethnic and Civilizational Conflicts,"Comparison of dehumanization in Israeli-Palestinian and Ukrainian-Russian conflicts, and Huntington's clash of civilizations theory."
US Imperialism and Global Resistance,"US efforts to control global trade, monetary systems, and regime changes, provoking fear and resistance worldwide."
Global Civilizational Divide,"Potential fracture between NATO/US and the rest of the world, focusing on control over resources and industrial power."
Internal Political Struggles in the US and Europe,"Populism, political polarization, and the struggle over narratives and economic policies within Western countries."
Narrative and Language in Politics,Importance of alternative narratives and statistics in shaping political upheavals and public understanding.
Immigration Discourse and Political Rhetoric,"Trump's extreme anti-immigrant rhetoric contrasted with softer responses, illustrating shifting political narratives."


### Table of hat perspectives

In [33]:
my $sophFeed = 
    llm-synthesize(
        llm-prompt("SophisticatedFeedback")($txtFocus, 'HTML'), 
        e => $conf41-mini
    );

$sophFeed = $sophFeed.subst('```html').subst('```');

$sophFeed.&text-stats

(chars => 8795 words => 1127 lines => 108)

In [34]:
#% html
$sophFeed

Hat Name,Perspective,Feedback,New Ideas
White Hat,Information and Facts,"The analysis accurately reflects current economic data: Germany’s industrial sector is under severe strain due to high energy costs and the loss of cheap Russian gas. Volkswagen’s announcement to cut employment is a concrete example of this distress. The geopolitical facts about U.S. sanctions, the global shift in power towards BRICS countries, and the political fragmentation in Europe are well documented. However, the complexity of global supply chains and the multifaceted nature of international relations mean these trends are dynamic and evolving.",1. Collect updated data on European industrial output and employment trends post-sanctions. 2. Analyze comparative EV market shares between China and Europe to quantify competitiveness gaps. 3. Monitor shifts in global trade patterns involving BRICS and Western economies.
Black Hat,Judgement and Caution,"The idea highlights many risks: irreversible industrial decline, geopolitical vulnerability, and political fragmentation. However, it may underestimate the resilience and adaptability of European economies and political systems. The assumption that firms like Volkswagen cannot be revived is pessimistic; history shows industrial recovery is possible with strategic investment. The narrative also risks fatalism, which could discourage proactive policy responses or innovation. Additionally, the portrayal of the U.S. as solely responsible for Europe’s woes oversimplifies complex international dynamics.",1. Identify specific barriers to industrial recovery and propose mitigation strategies. 2. Assess risks of escalating geopolitical conflicts affecting European infrastructure. 3. Develop contingency plans for supply chain disruptions and energy shortages.
Gray Hat,Cynicism and Skepticism,"The narrative reads like a critique of Western propaganda and self-justifying elites. The demonization of leftist politics and the portrayal of geopolitical conflicts as civilizational struggles could be seen as ideological framing to maintain existing power structures. The idea that the West is “losing” and resorting to chaos as a strategy might be an exaggeration to justify aggressive foreign policies or internal repression. There is a sense that economic and political elites are playing a game of survival, using narratives to distract from systemic failures and inequalities.",1. Investigate who benefits from maintaining the current narratives and conflicts. 2. Explore alternative media or academic sources that challenge mainstream geopolitical discourse. 3. Analyze the role of corporate lobbying in shaping both Western and Chinese economic policies.
Red Hat,Feelings and Emotion,"The idea evokes a sense of urgency, fear, and frustration. There is palpable anxiety about the loss of industrial jobs, the fracturing of societies, and the rise of nationalism and authoritarianism. The emotional tone suggests despair over the inability to find clear solutions and a feeling of betrayal by political elites. It also stirs anger about perceived injustices, such as demonizing immigrants or left-wing movements, and a deep sadness about the potential decline of European civilization as it has been known.",1. Foster public dialogues that acknowledge these fears and frustrations openly. 2. Create community initiatives to support displaced workers and marginalized groups. 3. Promote narratives of hope and resilience to counteract despair and fatalism.
Yellow Hat,Benefits and Creativity,"Despite the challenges, this situation could catalyze innovation and transformation. The pressure on European industries might accelerate investment in green technologies and renewable energy, reducing dependency on geopolitically risky suppliers. The political fragmentation could open space for new, inclusive political movements that better represent diverse interests. The global shift in power might encourage more multipolar cooperation frameworks, potentially

### Summaries or insights

In [36]:
my $res = llm-synthesize([ 
    llm-prompt('FindPropagandaMessage'),
    'TEXT START:',
    $txtFocus,
    'TEXT END'
],
e => $conf41-mini,
);

$res.&text-stats

(chars => 8649 words => 1168 lines => 88)

In [37]:
#% markdown
$res.subst( / ^^ '## '/, '#### ', :g).subst( / ^^ '# '/, '### ', :g)

OVERT MESSAGE  
Europe and Germany face irreversible economic decline due to sanctions and geopolitical struggles with the US.

HIDDEN MESSAGE  
US-led sanctions and global power plays intentionally destroy European industry to maintain American global dominance.

HIDDEN OPINIONS  
- Western elites prioritize US hegemony over allied economic well-being and sovereignty.  
- Sanctions are weaponized to suppress competitors, not to promote justice or peace.  
- European industrial decline is a deliberate consequence of US geopolitical strategy.  
- Left-wing parties are co-opted or misrepresented to weaken genuine opposition.  
- The US uses chaos and conflict to maintain global control amid decline.  
- Global economic order enforces neoliberal policies favoring Western corporate interests.  
- BRICS nations represent a rising counterforce challenging Western imperialism.  
- Media and academia systematically exclude alternative economic narratives to preserve status quo.  
- Debt and currency systems are manipulated to enrich elites and suppress populations.  
- Military and economic coercion replace traditional warfare to control other nations.  
- Western democracies increasingly undermine true democracy to enforce US foreign policy.  
- Corporate interests often oppose nationalist or populist movements when inconvenient.  
- Climate policy debates mask economic competition and power struggles.  
- The West’s decline is inevitable but denied publicly to maintain control.  
- Terrorism and corruption are tools used by Western powers to destabilize rivals.  
- Global population disparities render Western military dominance obsolete.  
- The US neoliberal “religion” suppresses alternative development models globally.  
- Political language is manipulated to conceal systemic economic and geopolitical failures.  
- The current crisis is a symptom of deeper systemic contradictions in capitalism.

SUPPORTING ARGUMENTS and QUOTES  
- Volkswagen’s layoffs and industry decline illustrate sanctions’ economic damage: “Volkswagen said…cut back employment…not able to compete in electric cars with China.”  
- Putin’s threat to bomb factories signals real geopolitical risks from Western policies.  
- “America has created irreversibilities for Europe while maintaining its own freedom.”  
- The demonization of left-wing parties and socialists as threats recalls Cold War propaganda: “the Communists are the great danger.”  
- The narrative of “chaos” as natural market functioning masks deliberate destabilization strategies.  
- Debt is reframed as a financial asset for elites, not a burden for society: “US debt is perhaps the most desirable financial asset.”  
- Corporate leaders’ continued investment in China reveals internal contradictions in Western policy.  
- The G7’s small population share versus BRICS’ large population highlights shifting global power.  
- Suspension and nullification of elections in NATO countries demonstrate erosion of democratic norms.  
- The media’s silence on alternative economic models and statistics supports the dominant narrative.  
- Climate policies are entangled with economic competition rather than pure environmental concerns.  
- Terrorism and corruption are described as Western strategies to maintain influence without direct warfare.  
- The US and Europe’s refusal to buy Russian energy and the resulting economic fallout reveal shortsightedness or deliberate sabotage.  
- The framing of Palestinians and Arabs as subhuman echoes Nazi ideology, showing ideological continuity in Western propaganda.  
- The description of the US neoliberal system as a “religion” signals ideological rigidity suppressing dissent.  
- The discussion of populism as a response to ruling class failures highlights political instability.

DESIRED AUDIENCE OPINION CHANGE  
- Believe US sanctions harm allies more than enemies.  
- Distrust mainstream economic and political narratives about globalization.  
- View left-wing parties as compromised or misrepresented by elites.  
- Recognize BRICS as legitimate challengers to Western dominance.  
- See Western democracy as increasingly authoritarian and manipulative.  
- Understand debt as a tool for elite enrichment, not public good.  
- Accept that climate debates mask deeper economic power struggles.  
- Be skeptical of media and academic exclusion of alternative views.  
- Perceive current geopolitical conflicts as struggles for global economic control.  
- Question the morality and effectiveness of US-led foreign policy.

DESIRED AUDIENCE ACTION CHANGE  
- Support political movements opposing neoliberal economic policies.  
- Demand transparency and alternative economic data from media and institutions.  
- Resist acceptance of US-led sanctions harming local economies.  
- Advocate for stronger ties with BRICS and non-Western alliances.  
- Challenge narratives demonizing left-wing and socialist parties.  
- Promote policies protecting domestic industries from foreign economic warfare.  
- Oppose military interventions justified by distorted security threats.  
- Encourage democratic reforms preventing election interference by NATO powers.  
- Support labor unions and industrial workers facing economic decline.  
- Push for climate policies aligned with economic justice and sovereignty.

MESSAGES  
The speaker wants you to believe he is discussing European economic challenges caused by war, but he is actually exposing US-led strategies deliberately undermining European sovereignty and industry.

PERCEPTIONS  
The speaker wants you to believe he is a neutral analyst concerned about global stability, but he’s actually a cynical critic exposing Western imperial hypocrisy and systemic economic manipulation.

ELLUL'S ANALYSIS  
According to Jacques Ellul, propaganda works by simplifying complex realities into emotionally charged narratives that create a collective attitude. This speaker uses a complex geopolitical and economic analysis to reveal how Western propaganda masks economic warfare as moral crusades. The repeated demonization of left-wing groups and the framing of chaos as market naturalness are classic propaganda techniques shaping public attitudes to accept neoliberal dominance and military interventions. The speaker’s critique exposes the constructed nature of these narratives, showing how propaganda serves to perpetuate elite power and suppress alternative voices.

BERNAYS' ANALYSIS  
Edward Bernays emphasized the engineering of consent through controlling information and shaping public opinion. This discourse reveals how elites manufacture consent for harmful policies by demonizing political opponents, controlling economic narratives, and manipulating language around democracy and security. The speaker highlights how media and academic institutions exclude alternative perspectives, maintaining elite dominance. The internal contradictions between corporate interests and political rhetoric demonstrate the subtle conflicts Bernays described, where public opinion is engineered to accept policies serving narrow elite interests under the guise of national security and economic progress.

LIPPMANN'S ANALYSIS  
Walter Lippmann argued that the public relies on simplified “pictures in their heads” created by elites to make sense of complex realities. This speaker critiques these manufactured images, revealing how Western powers create false narratives about democracy, economic competition, and security to justify their actions. The demonization of leftists and the framing of economic decline as unavoidable chaos are examples of these simplified pictures. Lippmann’s insight shows how these narratives limit genuine public understanding and debate, trapping populations in illusions that serve elite agendas rather than their own interests.

FRANKFURT'S ANALYSIS  
Harry Frankfurt’s concept of “bullshit” applies to the speaker’s portrayal of political and economic narratives that disregard the truth to manipulate audiences. The speaker exposes how elites produce discourse indifferent to factual accuracy, focusing instead on persuading people to accept harmful policies. The shifting meanings of language—chaos as market naturalness, democracy as support for US policy—are examples of “bullshit” that obscure reality. This manipulation erodes public trust and rational debate, demonstrating how political discourse often prioritizes power maintenance over honest communication.

NOTE: This AI is tuned specifically to be cynical and politically-minded. Don't take it as perfect. Run it multiple times and/or go consume the original input to get a second opinion.

-------

## References

### Articles

[AA1] Anton Antonov, 
["Outlier detection in a list of numbers"](https://rakuforprediction.wordpress.com/2022/05/29/outlier-detection-in-a-list-of-numbers/),
(2022),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

### Packages

[AAp1] Anton Antonov,
[WWW::OpenAI, Raku package](https://github.com/antononcube/Raku-WWW-OpenAI),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov,
[WWW::PaLM, Raku package](https://github.com/antononcube/Raku-WWW-PaLM),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov,
[LLM::Functions, Raku package](https://github.com/antononcube/Raku-LLM-Functions),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp4] Anton Antonov,
[LLM::Prompts, Raku package](https://github.com/antononcube/Raku-LLM-Prompts),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp5] Anton Antonov,
[ML::FindTextualAnswer, Raku package](https://github.com/antononcube/Raku-ML-FindTextualAnswer),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp6] Anton Antonov,
[Math::Nearest, Raku package](https://github.com/antononcube/Raku-Math-Nearest),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp7] Anton Antonov,
[Math::DistanceFunctions, Raku package](https://github.com/antononcube/Raku-Math-DistanceFunctions),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp8] Anton Antonov,
[Statistics::OutlierIdentifiers, Raku package](https://github.com/antononcube/Raku-Statistics-OutlierIdentifiers),
(2022),
[GitHub/antononcube](https://github.com/antononcube).

## Videos

[AAv1] Anton Antonov,
["Raku RAG demo"](https://www.youtube.com/watch?v=JHO2Wk1b-Og),
(2024),
[YouTube/AAA4prediction](https://www.youtube.com/@AAA4prediction).